In [1]:
import Ipynb_importer
from a_basic_quant import *
from b_model import *
from c_train_and_test import *
from d_post_training_quantize import *

importing Jupyter notebook from a_basic_quant.ipynb
importing Jupyter notebook from b_model.ipynb
importing Jupyter notebook from c_train_and_test.ipynb
importing Jupyter notebook from d_post_training_quantize.ipynb


In [2]:
a = torch.randn((64,1,28,28))
a.shape

torch.Size([64, 1, 28, 28])

In [3]:
conv = torch.nn.Conv2d(1, 40, 3, 1)
res = conv(a)

In [4]:
min_a, max_a = calcu_max_and_min(a, None, None, False)
scale_a, zero_point_a = calcu_scale_and_zeropoint(min_a, max_a, 8, False)
q_a = quantize_tensor(a, scale_a, zero_point_a).int()
dq_a = dequantize_tensor(q_a, scale_a, zero_point_a)
print(a[0][0:5][0][0])
print(dq_a[0][0:5][0][0])

tensor([ 0.7126,  0.1340,  1.4468,  0.1608, -0.4743,  0.5212, -1.5438, -0.0798,
         1.3271,  0.2168, -1.1680, -1.3196, -0.4271,  0.2910,  0.8130,  0.7308,
        -1.0762,  0.5525, -0.2756, -0.4258,  0.0287, -0.8183, -1.3653, -1.5111,
         1.0644,  1.3089, -0.4093, -0.2931])
tensor([ 0.6831,  0.1242,  1.4283,  0.1552, -0.4968,  0.4968, -1.5525, -0.0931,
         1.3041,  0.1863, -1.1799, -1.3351, -0.4347,  0.2794,  0.8073,  0.7141,
        -1.0867,  0.5278, -0.2794, -0.4347,  0.0000, -0.8383, -1.3662, -1.5214,
         1.0557,  1.3041, -0.4347, -0.3105])


In [5]:
w = conv.weight.data
min_w, max_w = calcu_max_and_min(w, None, None, True)
scale_w, zero_point_w = calcu_scale_and_zeropoint(min_w, max_w, 8, True)
q_w = quantize_tensor(w, scale_w, zero_point_w,8,False, True)
dq_w = dequantize_tensor(q_w, scale_w, zero_point_w, True)
print(w[0][0:5][0][0])
print(dq_w[0][0:5][0][0])

tensor([ 0.0546,  0.2338, -0.3169])
tensor([ 0.0546,  0.2338, -0.3148])


In [6]:
min_res, max_res = calcu_max_and_min(res, None, None, False)
scale_res, zero_point_res = calcu_scale_and_zeropoint(min_res, max_res, 8, False)
q_res = quantize_tensor(res, scale_res, zero_point_res).int()
dq_res = dequantize_tensor(q_res, scale_res, zero_point_res)
print(res[0][0:5][0][0])
print(dq_res[0][0:5][0][0])

tensor([-0.2963,  1.0422, -0.0454, -0.0931,  0.2671, -0.8507, -0.4561, -0.1652,
         0.3485, -0.2223,  0.1644, -0.5918,  0.3604,  0.1135,  0.9989, -0.1687,
         0.8062, -0.0770,  0.5284, -0.4346,  0.6680,  0.5878, -0.9849,  0.3461,
         0.7483,  0.0630], grad_fn=<SelectBackward>)
tensor([-0.3053,  1.0178, -0.0509, -0.1018,  0.2544, -0.8651, -0.4580, -0.1781,
         0.3308, -0.2290,  0.1527, -0.6107,  0.3562,  0.1018,  0.9923, -0.1781,
         0.7888, -0.1018,  0.5089, -0.4580,  0.6616,  0.5852, -0.9923,  0.3308,
         0.7379,  0.0509])


In [7]:
q_w[0].view(1,1,3,3).shape

torch.Size([1, 1, 3, 3])

In [8]:
conv.bias[0].shape

torch.Size([])

In [9]:
x = q_a - zero_point_a
y = F.conv2d(input = x, weight = q_w[0].view(1,1,3,3), bias=conv.bias[0].view(1),\
            stride=conv.stride,
            padding=conv.padding,
            dilation=conv.dilation,
            groups=conv.groups)
y.shape

torch.Size([64, 1, 26, 26])

In [10]:
conv.weight.data.shape

torch.Size([40, 1, 3, 3])

In [11]:
q_w.shape

torch.Size([40, 1, 3, 3])

In [12]:
torch.tensor(zero_point_w)[23]

tensor(117., dtype=torch.float64)

In [13]:
for i in range(len(zero_point_w)):
    conv.weight.data[i] = q_w[i] - zero_point_w[i]

In [14]:
x_2 = q_a - zero_point_a
y_2 = conv(x_2)
y_2.shape

torch.Size([64, 40, 26, 26])

In [15]:
M = [scale_w[i] * scale_a / scale_res for i in range(len(scale_w))]
len(M)

40

In [16]:
for i in range(len(M)):
    y_2[:,i,:,:] = y_2[:,i,:,:] * M[i]
y_2[0][0:5][0][0]

tensor([-16.5199,  36.1009,  -6.6106,  -8.5260,   5.2803, -38.1270, -22.8699,
        -11.2940,   8.4803, -14.0009,   1.4630, -28.0986,   8.9416,  -0.3656,
         34.0861, -11.2549,  26.3759,  -8.0280,  16.1838, -22.1394,  20.9141,
         18.2061, -43.3648,   8.4468,  24.7519,  -2.5970],
       grad_fn=<SelectBackward>)

In [17]:
z_2 = y_2 +zero_point_res
z_2[0][0:5][0][0]

tensor([114.4801, 167.1010, 124.3894, 122.4740, 136.2803,  92.8730, 108.1301,
        119.7060, 139.4803, 116.9991, 132.4630, 102.9014, 139.9416, 130.6344,
        165.0862, 119.7451, 157.3759, 122.9720, 147.1837, 108.8606, 151.9141,
        149.2061,  87.6352, 139.4468, 155.7519, 128.4030],
       grad_fn=<SelectBackward>)

In [18]:
dz_2 = dequantize_tensor(z_2, scale_res, zero_point_res)
dz_2[0][0:5][0][0]

tensor([-0.4203,  0.9186, -0.1682, -0.2169,  0.1344, -0.9701, -0.5819, -0.2874,
         0.2158, -0.3562,  0.0372, -0.7150,  0.2275, -0.0093,  0.8673, -0.2864,
         0.6711, -0.2043,  0.4118, -0.5633,  0.5321,  0.4632, -1.1034,  0.2149,
         0.6298, -0.0661], grad_fn=<SelectBackward>)

In [19]:
a = torch.randn((64,1,28,28))
a.shape

torch.Size([64, 1, 28, 28])

In [20]:
conv = torch.nn.Conv2d(1, 40, 3, 1)
b = conv(a)

In [21]:
qconv = QConv2d(conv, True, True, 8, True)

In [22]:
q_b = qconv(a)

In [23]:
qconv.freeze()

In [24]:
q_x = qconv.q_in.quantize_tensor(a)
q_x = qconv.quantize_inference(q_x)

In [25]:
out = qconv.q_out.dequantize_tensor(q_x)
out[0][0][:5][0]

tensor([-0.5812, -0.7556, -0.3778,  0.0581, -0.1744,  0.6103,  0.2906,  0.4940,
         0.0581, -0.1453, -0.1453,  0.3487,  1.0462,  0.1453, -0.4940, -0.4650,
         0.1453, -0.1453, -0.7265, -0.0291,  0.6394,  0.9590,  0.6684,  0.1453,
         0.1453,  1.0172], grad_fn=<SelectBackward>)

In [26]:
b[0][0][:5][0]

tensor([-0.5767, -0.7625, -0.3758,  0.0535, -0.1850,  0.6198,  0.2780,  0.4998,
         0.0678, -0.1520, -0.1324,  0.3410,  1.0518,  0.1388, -0.4951, -0.4791,
         0.1440, -0.1452, -0.7232, -0.0254,  0.6318,  0.9714,  0.6540,  0.1549,
         0.1344,  1.0298], grad_fn=<SelectBackward>)

In [42]:
a = torch.randn((64,1,28,28)).cuda()
a[0][0][0][0]-1

tensor(-1.4769, device='cuda:0')

In [36]:
conv = torch.nn.Conv2d(1, 40, 3, 1)
conv.cuda()
b = conv(a)

In [39]:
qconv = QConv2d(conv, True, True, 8, True)
qconv.cuda()

QConv2d(
  (conv_module): Conv2d(1, 40, kernel_size=(3, 3), stride=(1, 1))
)

In [41]:
q_b = qconv(a)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
qconv.freeze()

In [ ]:
q_x = qconv.q_in.quantize_tensor(a)
q_x = qconv.quantize_inference(q_x)

In [ ]:
out = qconv.q_out.dequantize_tensor(q_x)
out[0][0][:5][0]

In [ ]:
b[0][0][:5][0]